In [1]:
from autogluon.tabular import TabularPredictor, TabularDataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

/Users/bello/anaconda3/envs/tomato/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### data preprocess
# input data import
input_data = pd.read_csv('2023_smartFarm_AI_hackathon_dataset.csv')

# 데이터셋을 data, target으로 변수분리
data = input_data.iloc[:,:-1]
target = input_data.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

In [11]:
### RMSE, R2 evaluateß
# AutoGluon 모델을 저장
model_path = "AutogluonModels/default_1hr" # RMSE trained model

# 모델을 불러오기
loaded_predictor = TabularPredictor.load(model_path)

# 테스트 데이터 준비
test_df = TabularDataset(X_test) 
prediction = loaded_predictor.predict(test_df)

# RMSE 계산
rmse = np.sqrt(mean_squared_error(y_test, prediction))
r2score = r2_score(y_test, prediction)

In [13]:
### OUTPUT ###
print("RMSE:", rmse)
print("R2_score:", r2score)

RMSE: 3615.5786537397207
R2_score: 0.9999623472368988
